In [1]:
import os
import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)
print('Version:',tf.__version__)

1 Physical GPUs, 1 Logical GPUs
Version: 2.3.1


In [3]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [4]:
model.load_weights('checkpoints/my_checkpoint')

In [5]:
from imutils.video import VideoStream
import imutils
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np

In [6]:
# vs=VideoStream(src=0).start()
# frame=vs.read()
# frame=cv2.resize(frame,(300,300))
# vs.stop()
# vs.stream.release()
# frame.shape

In [7]:
# plt.imshow(frame)

In [8]:
def get_label(frame):
    x = image.img_to_array(frame)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    if classes[0]>0.5:
        label = "Human"
    else:
        label = "Horse"
    return label
# print(get_label(frame))

In [9]:
# label = get_label(frame)
# color=(0,255,0) if label=='Human' else (0,0,255)
# cv2.putText(frame,label,(25,25),cv2.FONT_HERSHEY_SIMPLEX,0.75,color,2)
# plt.imshow(frame)

In [11]:
vs=VideoStream(src=0).start()

while True:
    frame=vs.read()
    frame=cv2.resize(frame,(300,300))
    
    label = get_label(frame)
    color=(0,255,0) if label=='Human' else (0,0,255)
    cv2.putText(frame,label,(25,25),cv2.FONT_HERSHEY_SIMPLEX,0.75,color,2)

    #show the output frame
    cv2.imshow("Frame",frame)
    key=cv2.waitKey(1) & 0xFF
    
    if key==ord('q'):
        break
cv2.destroyAllWindows()
vs.stop()
vs.stream.release()